In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from tracking.filters.jipdaf import PDA, Track, build_clusters, track_betas
from tracking.filters.dynamics_models import DefaultDynamicsPV2D, DefaultDynamicsPVA2D
from tracking.util.interpolator import LinearInterpolator, NodeCollection, SplineInterpolator
from tracking.util.metrics import RadarGenerator, Scan, TrackLog, PoissonClutter
from tracking.util.path import Path2D, PathFactory
from tracking.visualizer import TrackVisualizer
from tracking.filters.track_manager import TrackManager, TrackingVisualizer

%load_ext autoreload
%autoreload 2

TMAX = 500

In [ ]:
nodes = [
    NodeCollection(np.array([[-400, -400],  [400, -400]]), [0, TMAX]),
    NodeCollection(np.array([[-400, -375],  [400, -375]]), [0, TMAX]),
    #NodeCollection(np.array([[-400, -200],  [400, -200]]), [0, TMAX]),
    #NodeCollection(np.array([[-400, -100],  [400, -100]]), [0, TMAX]),
    NodeCollection(np.array([[-400, -0],  [400, -0]]), [0, TMAX]),
    #NodeCollection(np.array([[-400, 100],  [400, 100]]), [0, TMAX]),
    NodeCollection(np.array([[-400, 200],  [400, 200]]), [0, TMAX]),
    #NodeCollection(np.array([[-400, 300],  [400, 300]]), [0, TMAX]),
    #NodeCollection(np.array([[-400, 400],  [400, 400]]), [0, TMAX]),
]

factory=PathFactory()
paths=factory.create(nodes, LinearInterpolator)

In [ ]:

nodes = [
    NodeCollection(np.array([[-400, 100],  [400, -100]]), [0, TMAX]),
]

node = NodeCollection(np.array(
    [[-400, -100], [-200, 25], [0, 0], [200, -25], [400, 100]]), np.linspace(0, TMAX, 5))
node2 = NodeCollection(np.array(
    [[400, 100], [200, -25], [0, 0], [-200, 25], [-400, -100]]), np.linspace(0, TMAX, 5))
node3 = NodeCollection(np.array(
    [[200, 300], [200, -25], [10, 5], [-200, 35], [-400, -100]]), np.linspace(0, TMAX, 5))
factory = PathFactory()
paths = factory.create(nodes, LinearInterpolator)
paths += factory.create([node, node2, node3], SplineInterpolator)

paths += factory.create([
    NodeCollection(np.array([
        [-400, 400], [-200, 300], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),

    NodeCollection(np.array([
        [-200, 300], [-100, 100], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),

    NodeCollection(np.array([
        [-200, 300], [-100, 180], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),

    NodeCollection(np.array([
        [200, 300], [100, 180], [0, 400], [-200, -300], [-400, -200]
    ]), np.linspace(100, TMAX, 5)),
], SplineInterpolator)

In [ ]:
nprobes = 200
probexy = np.zeros(nprobes)
probetimes = np.linspace(0.0, TMAX-0.1, nprobes)
radar = RadarGenerator(paths, 0.6, 1)
clutter_model = PoissonClutter([-400, 400, -220, 450], 1e-4)
scans = radar.make_scans_series(
    np.array([probetimes, probexy, probexy]).T, clutter_model)

In [ ]:

manager = TrackManager(DefaultDynamicsPVA2D())

nsteps = 60000
time = 0
dt = 0.05

working_scans = scans.copy()

for _ in range(nsteps):
    time += dt

    manager.predict_tracks(time, dt)

    if time+dt >= working_scans[0].time:
        time = working_scans[0].time

        manager.update_tracks(working_scans[0], time)
        manager.one_point_init(vmax=5, p0=0.25, disable_clutter_tracks=False)
        manager.delete_false_tracks(.005)

        working_scans.pop(0)
        if len(working_scans) == 0:
            print("Processed all scans")
            break

In [ ]:
%autoreload 2
%matplotlib qt 

renderer = TrackingVisualizer(manager.logger)
plt.ion()
plt.figure()
plot = plt.subplot(1, 2, 1)
    
plot2 = plt.subplot(1, 2, 2)
renderer.filter_tracks_by_length(7)
renderer.render_epochs_in_range(plot2, -1, -1)
# renderer.set_number_rendered_epochs(0)
# renderer.render(plot2)

renderer.set_number_rendered_epochs(25)
renderer.clear_filter()
renderer.filter_tracks_by_length(7)

i = 0
render_interval = 3

while renderer.advance_epoch():
    i += 1
    if i % render_interval != 0:
        continue
    plot.cla()

    plot.set_ylim(-450, 450)
    plot.set_xlim(-450, 450)

    #renderer.render_measurements(plot)
    renderer.render(plot)
    plt.show(block=False)
    plt.pause(0.0001)


In [ ]:
track_lengths = manager.logger.track_lengths()
track_lengths_filtered = manager.logger.display_track_lengths(5)

for t in track_lengths_filtered:
    exst_prob_hist = manager.logger.track_existence_probability_history(t)
    plt.plot(exst_prob_hist)

plt.show()

In [ ]:
track_uid = 696
plt.figure()
ms_plot = plt.subplot(2, 2, (1))
plt.grid(True)
measurements_data = manager.logger.track_selected_measurements_history(track_uid)
line1, = ms_plot.plot([ep['num_actual_measurements'] for ep in measurements_data.values()], label='actual')
line2, = ms_plot.plot([ep['num_clutter_measurements'] for ep in measurements_data.values()], label='clutter')
plt.legend(['actual', 'clutter'])
exst_plot = plt.subplot(2, 2, (3))
plt.grid(True)
track_plot = plt.subplot(2, 2, (2,4))
plt.show()

exst_plot.plot(manager.logger.track_existence_probability_history(track_uid))
renderer.render_track(track_plot, track_uid)
plt.show()


In [ ]:
plt.hist(track_lengths, bins=200);
plt.show()

In [ ]:
plt.figure()
for path in paths:
    trange = np.linspace(path.t_min, path.t_max, 300)
    acc = [path.acc(t) for t in trange]
    plt.plot(acc)